# 7章 畳み込みニューラルネットワーク
本章のテーマは、畳み込みニューラルネットワーク(convolutional neural network:CNN)です。CNN は、画像認識や音声認識など、至るところで使わ れています。また、画像認識のコンペティションでは、ディープラーニングによる手 法のほとんどすべてが CNN をベースとしています。本章では、CNN のメカニズム について詳しく説明し、その処理内容を Python で実装していきます。

## 7.1 全体の構造
まずは CNN の大枠を理解するために、CNN のネットワーク構造から見ていくこ とにします。CNN も、これまで見てきたニューラルネットワークと同じで、レゴブ ロックのようにレイヤを組み合わせて作ることができます。ただし CNN の場合、新 たに「Convolution レイヤ(畳み込み層)」と「Pooling レイヤ(プーリング層)」が 登場します。畳み込み層とプーリング層の詳細は次節以降で説明するとして、ここで は、どのようにレイヤを組み合わせて CNN が構築されるかを先に見ていきます。

CNN では、新しく「Convolution レイヤ」と「Pooling レイヤ」が加わります。CNN のレイヤのつながり順は、「Convolution - ReLU - (Pooling)」という流れです(Pooling レイヤは省略されることもあります)。これは、 今までの「Affine - ReLU」というつながりが、「Convolution - ReLU - (Pooling)」 に置き換わったと考えることができます。
CNN で他に注目する点は、出力に近い層では、これまでの「Affine - ReLU」という組み合わせが用いられるということです。また、最後の出力層におい ては、これまでの「Affine - Softmax」の組み合わせが用いられます。これらは一般 的な CNN でよく見られる構成です。

## 7.2 畳み込み層
CNN では、パディング、ストライドなどの CNN 特有の用語が登場します。また、 各層を流れるデータは形状のあるデータ(たとえば、3 次元のデータ)になり、これまでの全結合のネットワークとは異なります。そのため、初めて CNN を学ぶとき は、分かりにくく感じるかもしれません。ここでは、CNN で使われる畳み込み層の 仕組みを、じっくりと時間をかけて見ていきたいと思います。

### 7.2.1 全結合層の問題点
これまで見てきた全結合のニューラルネットワークでは、全結合層(Affine レイ ヤ)を用いました。全結合層では、隣接する層のニューロンがすべて連結されてお り、出力の数は任意に決めることができます。<br>
全結合層の問題点は何でしょうか。それは、データの形状が“無視”されてしまう ことです。たとえば入力データが画像の場合、画像は通常、縦・横・チャンネル方向 の 3 次元の形状です。しかし、全結合層に入力するときには、3 次元のデータを平ら ―― 1 次元のデータ――にする必要があります。実際、これまでの MNIST データ セットを使った例では、入力画像は (1, 28, 28)―― 1 チャンネル、縦 28 ピクセル、 横 28 ピクセル――の形状でしたが、それを 1 列に並べた 784 個のデータを最初の Affine レイヤへ入力しました。<br>

一方、畳み込み層(Convolution レイヤ)は、形状を維持します。画像の場合、入 力データを 3 次元のデータとして受け取り、同じく 3 次元のデータとして、次の層に データを出力します。そのため、CNN では、画像などの形状を有したデータを正し く理解できる(可能性がある)のです。<br>
なお、CNN では、畳み込み層の入出力データを、特徴マップ(feature map)と 言う場合があります。さらに、畳み込み層の入力データを入力特徴マップ(input feature map)、出力データを出力特徴マップ(output feature map)と言います。 本書では、「入出力データ」と「特徴マップ」を同じ意味の言葉として用います。

### 7.2.2 畳み込み演算
畳み込み層で行う処理は「畳み込み演算」です。畳み込み演算は、画像処理で言う ところの「フィルター演算」に相当します。<br>
図7-3 で示すように、畳み込み演算は、入力データに対して、フィルターを適用し ます。この例では、入力データは縦・横方向の形状を持つデータで、フィルターも同 様に、縦・横方向の次元を持ちます。データとフィルターの形状を、(height, width) で表記するとして、この例では、入力サイズは (4, 4)、フィルターサイズは (3, 3)、出 力サイズは (2, 2) になります。なお、文献によっては、ここで述べたような「フィル ター」という用語は、「カーネル」という言葉で表現されることもあります。<br>
<img src="./fig/スクリーンショット 2022-08-18 19.29.22.png">

畳み込み演算は、入力データに対して、フィルターのウィンドウを一定の間隔でス ライドさせながら適用していきます。ここで言うウィンドウとは、図7-4 における灰 色の 3 × 3 の部分を指します。図7-4 に示すように、それぞれの場所で、フィルター の要素と入力の対応する要素を乗算し、その和を求めます(この計算を積和演算と呼 ぶこともあります)。そして、その結果を出力の対応する場所へ格納していきます。 このプロセスをすべての場所で行うことで、畳み込み演算の出力を得ることができ ます。<br>
<img src="./fig/スクリーンショット 2022-08-18 19.34.31.png">

畳み込み演算のバイアス<br>
<img src="./fig/スクリーンショット 2022-08-18 20.27.52.png"><br>
図7-5 に示すように、バイアス項の加算は、フィルター適用後のデータに対して行 われます。ここで示すように、つねにバイアスはひとつ(1 × 1)だけ存在します(こ の例では、フィルター適用後のデータ 4 つに対してバイアスはひとつです)。そのひ とつの値がフィルター適用後のすべての要素に加算されます。

### 7.2.3 パディング
畳み込み層の処理を行う前に、入力データの周囲に固定のデータ(たとえば 0 な ど)を埋めることがあります。これをパディング(padding)と言って、畳み込み演 算ではよく用いられる処理です。たとえば、図 7 - 6 の例では、(4, 4) のサイズの入力 データに対して、幅 1 のパディングを適用しています。幅 1 のパディングとは、周囲 を幅 1 ピクセルの 0 で埋めることを言います。
<img src="./fig/スクリーンショット 2022-08-18 20.45.34.png"><br>
図7-6 に示すように、(4,4) のサイズの入力データはパディングによって、(6,6) の形状になります。そして、(3, 3) のサイズのフィルターをかけると、(4, 4) のサイ ズの出力データが生成されます。この例では、パディングを 1 に設定しましたが、パ ディングの値は 2 や 3 など任意の整数に設定することができます。もし図7-5 の例 でパディングを 2 に設定すれば、入力データのサイズは (8, 8) になり、パディングが 3 であれば、サイズは (10, 10) になります。

パディングを使う主な理由は、出力サイズを調整するためにあります。たとえ ば、(4, 4) のサイズの入力データに (3, 3) のフィルターを適用する場合、出力 サイズは (2, 2) になり、出力サイズは入力サイズから 2 要素分だけ縮小される ことになります。これは、畳み込み演算を何度も繰り返して行うようなディー プなネットワークでは問題になります。なぜなら、畳み込み演算を行うたびに 空間的に縮小されるのであれば、ある時点で出力サイズが 1 になってしまい、 それ以上畳み込み演算を適用できなくなるかもしれません。そのような事態を 回避するには、パディングを使用します。先の例では、パディングの幅を 1 に 設定すれば、入力サイズの (4, 4) に対して、出力サイズも (4, 4) のままサイズ は保たれます。そのため、畳み込み演算によって、空間的なサイズを一定にし たまま次の層へデータを渡すことができるのです。

### 7.2.4 ストライド
